In [1]:
import nltk
nltk.download('stopwords');
import time
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import collections
from numpy import linalg as la
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from collections import Counter
from config import *
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
import string
import unidecode
from wordcloud import WordCloud, ImageColorGenerator
import re
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words("english"))
ps = PorterStemmer()
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
 
import seaborn as sns
sns.set_style("darkgrid")

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import multiprocessing
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from nltk import bigrams
import time
from nltk.util import ngrams

[nltk_data] Downloading package stopwords to /Users/joan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
with open("one.json", "rb") as f:
    lines = f.readlines()
    lines = [json.loads(str_) for str_ in lines]

In [8]:
df_tweets = pd.DataFrame.from_records(lines)
df_tweets['h_field'] = df_tweets["entities"].apply(lambda x: x['hashtags'])
df_tweets['hashtags'] = df_tweets['h_field'].apply(lambda x: x[0]['text'] if x != [] else None)
df_tweets['retweeted_status'] = df_tweets['retweeted_status'].apply(lambda x: 1 if str(x)== 'nan' else 0)
df_tweets['id_tweet'] = df_tweets.index

data = df_tweets[df_tweets['retweeted_status'] == 1][['id_tweet', 'text']]
#data['id_tweet'] = str(data['id_tweet'])
data['join'] = data[['id_tweet', 'text']].apply(lambda x: '{}|{}'.format(x[0],x[1]), axis=1)

In [9]:
def remove_links(text):
    if text:
        return re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)
    
    # In case there is no text
    return ""

def remove_hashtags(text):
    if text:
        return re.sub(r'#\w+ ?', '', text)
    
    return ""
def remove_accents(text):
    if text:
        return unidecode.unidecode(text)
        

    # In case there is no text
    return ""

def remove_punctuation_marks(text):
    if text:
        translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
        new_text = text.translate(translator)
        return " ".join(new_text.split())
        
    # In case there is no text
    return ""

def text_to_lower_case(text):
    if text:
        return text.lower()
    
    # In case there is no text
    return ""

def remove_emojis(text):
    if text:
        # TODO: Remove emojis (tip: search for encode - decode)
        returnString = ""
        for character in text:
            try:
                character.encode("ascii")
                returnString += character
            except UnicodeEncodeError:
                returnString += ''
        text = " ".join(returnString.split())
        
    return text

    # In case there is no text
    return ""

def remove_multiple_whitespaces(text):
    if text:
        return " ".join(text.split())

    # In case there is no text
    return ""

def remove_text_marks(text):
    if text:
        text = text.translate(str.maketrans('', '', string.punctuation))
        
        # TODO: replace *, ?, ... by spaces
        
        return text.strip()
    
    # In case there is no text
    return ""

def split_text_and_numbers(text):
    temp = re.compile("([a-zA-Z]+)([0-9]+)") 
    try:
        res = temp.match(text).groups()
        text = " ".join(res)
    except:
        text = text
    return text

def remove_alone_numbers(text):
    if text:
        text = ' '.join(filter(lambda word:word.replace('.','').isdigit()==False, text.split()))
        
        return text
    
    return ""

def remove_stopwords(text):
    text = text.split()
    return " ".join([x for x in text if x not in STOPWORDS])

def stemming(text):
    text = text.split()
    return " ".join([ps.stem(x) for x in text])

def clean_text(text):
    # Apply the different functions in order to clean the text
    text = remove_links(text)
    text = remove_hashtags(text)
    text = text_to_lower_case(text)
    text = remove_text_marks(text)
    text = remove_accents(text)
    text = remove_emojis(text)
    text = split_text_and_numbers(text)
    #text = remove_alone_numbers(text)
    text = remove_multiple_whitespaces(text)
    text = remove_punctuation_marks(text)
    text = remove_stopwords(text)
    text = stemming(text)
    
    # Return
    return text.split()

In [10]:
def create_index(lines):
    """
    Impleent the inverted index
    
    Argument:
    lines -- collection of Wikipedia articles
    
    Returns:
    index - the inverted index (implemented through a python dictionary) containing terms as keys and the corresponding 
    list of document these keys appears in (and the positions) as values.
    """
    index=defaultdict(list) 
    titleIndex = {} # dictionary to map page titles to page ids
    for line in lines: # Remember, lines contain all documents, each line is a document
        line_arr = line.split("|")
        page_id = int(line_arr[0])
        terms = clean_text(line_arr[1]) #page_title + page_text
  ## we do not need to apply get terms to title because it used only to print titles and not in the index
        
        ## ===============================================================        
        ## create the index for the current doc and store it in termdictPage
        ## termdictPage ==> { ‘term1’: [currentdoc, [list of positions]], ...,‘termn’: [currentdoc, [list of positions]]}
        
        ## Example: if the curr_doc has id 1 and his text is 
        ## "web retrieval information retrieval":
        
        ## termdictPage ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,4]], ‘information’: [1, [2]]}
        
        ## the term ‘web’ appears in document 1 in positions 0, 
        ## the term ‘retrieval’ appears in document 1 in positions 1 and 4
        ## ===============================================================
        
        termdictPage={}

        for position, term in enumerate(terms): # terms contains page_title + page_text. Loop over all terms
            try:
                # if the term is already in the index for the current page (termdictPage)
                # append the position to the corrisponding list
                
        ## START CODE
                termdictPage[term][1].append(position)  
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                termdictPage[term]=[page_id, array('I',[position])] #'I' indicates unsigned int (int in python)
            
        #merge the current page index with the main index
        for termpage, postingpage in termdictPage.items():
            index[termpage].append(postingpage)
        
        ## END CODE                    
                    
    return index

In [11]:
arr = []
for line in data['join']:
    arr.append(line)
index = create_index(arr)

In [12]:
def search(query, index):
    '''
    The output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    '''
    query=clean_text(query)
    #docs=set()
    docs = set(np.linspace(0,len(df_tweets)-1, len(df_tweets)))
    for term in query:
    ## START DODE
        try:
            # store in termDocs the ids of the docs that contain "term"                        
            termDocs=[posting[0] for posting in index[term]]
            # docs = docs Union termDocs
            docs = docs.intersection(termDocs)
        except:
            #term is not in index
            pass
    docs=list(docs)
    return docs

In [13]:
print("Insert your query:\n")
query = input()
docs = search(query, index)    
top = 10

df_tweets.loc[docs][['text', 'user', 'created_at', 'hashtags' , 'favorite_count', 'retweet_count', 'source']]

Insert your query:

trump


,text,user,created_at,hashtags,favorite_count,retweet_count,source
0,"Interior Secretary David Bernhardt, a Colorado...","{'id': 993301872751529984, 'id_str': '99330187...",Thu Dec 17 14:41:35 +0000 2020,copolitics,0,0,"<a href=""https://about.twitter.com/products/tw..."
90116,Trump did EVERYTHING in his power to undermine...,"{'id': 828596979160797184, 'id_str': '82859697...",Thu Dec 17 15:23:12 +0000 2020,None,0,0,"<a href=""http://twitter.com/download/iphone"" r..."
81929,Ivanka Trump Faces Off With Father to See How ...,"{'id': 18722822, 'id_str': '18722822', 'name':...",Thu Dec 17 15:19:45 +0000 2020,None,0,0,"<a href=""https://mobile.twitter.com"" rel=""nofo..."
90127,That just makes it even better . Would they ev...,"{'id': 1235658301330919424, 'id_str': '1235658...",Thu Dec 17 15:23:12 +0000 2020,None,0,0,"<a href=""https://mobile.twitter.com"" rel=""nofo..."
204816,Online news matters: States that voted for Tru...,"{'id': 13138012, 'id_str': '13138012', 'name':...",Thu Dec 17 16:32:33 +0000 2020,None,0,0,"<a href=""https://mobile.twitter.com"" rel=""nofo..."
...,...,...,...,...,...,...,...
49138,Explains Trump's behavior...,"{'id': 249739297, 'id_str': '249739297', 'name...",Thu Dec 17 15:05:39 +0000 2020,None,0,0,"<a href=""http://twitter.com/download/android"" ..."
204788,@HMOTP @SenSchumer @realDonaldTrump I wasn't a...,"{'id': 1243393335592534017, 'id_str': '1243393...",Thu Dec 17 16:32:51 +0000 2020,None,0,0,"<a href=""http://twitter.com/download/android"" ..."
114677,@Bucshamsky @lestathavana @yashar @realDonaldT...,"{'id': 910139032306356226, 'id_str': '91013903...",Thu Dec 17 15:33:51 +0000 2020,None,0,0,"<a href=""https://mobile.twitter.com"" rel=""nofo..."
204791,@CNN Trump administration = anti human and not...,"{'id': 526211047, 'id_str': '526211047', 'name...",Thu Dec 17 16:32:50 +0000 2020,None,0,0,"<a href=""http://twitter.com/download/iphone"" r..."


In [20]:
def create_index_tfidf(lines, numDocuments):

        
    index=defaultdict(list)
    tf=defaultdict(list) #term frequencies of terms in documents (documents in the same order as in the main index)
    df=defaultdict(int)         #document frequencies of terms in the corpus
    titleIndex=defaultdict(str)
    idf=defaultdict(float)
    
    for line in lines: # Remember, lines contain all documents, each line is a document
        line_arr = line.split("|")
        page_id = int(line_arr[0])
        terms = clean_text(line_arr[1])         
        
        termdictPage={}

        for position, term in enumerate(terms): ## terms contains page_title + page_text
            try:
                # if the term is already in the dict append the position to the corrisponding list
                termdictPage[term][1].append(position)  
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                termdictPage[term]=[page_id, array('I',[position])] #'I' indicates unsigned int (int in python)
        
        #normalize term frequencies
        # Compute the denominator to normalize term frequencies (formula 2 above)
        # norm is the same for all terms of a document.
        norm=0
        for term, posting in termdictPage.items(): 
            # posting is a list containing doc_id and the list of positions for current term in current document: 
            # posting ==> [currentdoc, [list of positions]] 
            # you can use it to inferr the frequency of current term.
            norm+=len(posting[1])**2
        norm=math.sqrt(norm)


        #calculate the tf(dividing the term frequency by the above computed norm) and df weights
        for term, posting in termdictPage.items():     
            # append the tf for current term (tf = term frequency in current doc/norm)
            tf[term].append(np.round(len(posting[1])/norm,4))  ## SEE formula (1) above
            #increment the document frequency of current term (number of documents containing the current term)
            df[term]= df[term] + 1  # increment df for current term
        
        #merge the current page index with the main index
        for termpage, postingpage in termdictPage.items():
            index[termpage].append(postingpage)
            
            # Compute idf following the formula (3) above. HINT: use np.log
    for term in df:
        idf[term] = np.round(np.log(float(numDocuments/df[term])),4)
            
    return index, tf, df, idf, titleIndex


In [21]:
start_time = time.time()
numDocuments = len(data)
index, tf, df, idf, titleIndex = create_index_tfidf(data['join'], numDocuments)
print("Total time to create the index: {} seconds" .format(np.round(time.time() - start_time,2)))

Total time to create the index: 40.94 seconds


In [16]:

def rankDocuments(terms, docs, index, idf, tf, titleIndex):
    """
    Perform the ranking of the results of a search based on the tf-idf weights
    
    Argument:
    terms -- list of query terms
    docs -- list of documents, to rank, matching the query
    index -- inverted index data structure
    idf -- inverted document frequencies
    tf -- term frequencies
    titleIndex -- mapping between page id and page title
    
    Returns:
    Print the list of ranked documents
    """
        
    # I'm interested only on the element of the docVector corresponding to the query terms 
    # The remaing elements would became 0 when multiplied to the queryVector
    docVectors=defaultdict(lambda: [0]*len(terms)) # I call docVectors[k] for a nonexistent key k, the key-value pair (k,[0]*len(terms)) will be automatically added to the dictionary
    queryVector=[0]*len(terms)    

    # compute the norm for the query tf
    query_terms_count = collections.Counter(terms) # get the frequency of each term in the query. 
    # Example: collections.Counter(["hello","hello","world"]) --> Counter({'hello': 2, 'world': 1})
    # HINT: use when computing tf for queryVector
    
    query_norm = la.norm(list(query_terms_count.values()))
    
    for termIndex, term in enumerate(terms): #termIndex is the index of the term in the query
        if term not in index:
            continue
                    
        ## Compute tf*idf(normalize tf as done with documents)
        queryVector[termIndex]=query_terms_count[termIndex]/query_norm * idf[term] 

        # Generate docVectors for matching docs
        for docIndex, (doc, postings) in enumerate(index[term]):
            # Example of [docIndex, (doc, postings)]
            # 0 (26, array('I', [1, 4, 12, 15, 22, 28, 32, 43, 51, 68, 333, 337]))
            # 1 (33, array('I', [26, 33, 57, 71, 87, 104, 109]))
            # term is in doc 26 in positions 1,4, .....
            # term is in doc 33 in positions 26,33, .....
            
            #tf[term][0] will contain the tf of the term "term" in the doc 26            
            if doc in docs:
                docVectors[doc][termIndex]=tf[term][docIndex] * idf[term]  # TODO: check if multiply for idf

    # calculate the score of each doc
    # compute the cosine similarity between queyVector and each docVector:
    # HINT: you can use the dot product because in case of normalized vectors it corresponds to the cosine siilarity
    # see np.dot
    
    docScores=[ [np.dot(curDocVec, queryVector), doc] for doc, curDocVec in docVectors.items() ]
    docScores.sort(reverse=True)
    resultDocs=[x[1] for x in docScores]
    #print document titles instead if document id's
    #resultDocs=[ titleIndex[x] for x in resultDocs ]
    if len(resultDocs) == 0:
        print("No results found, try again")
        query = input()
        docs = search_tf_idf(query, index)    
    #print ('\n'.join(resultDocs), '\n')
    return resultDocs

In [17]:
def search_tf_idf(query, index):
    '''
    output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    '''
    query=clean_text(query)
    #docs=set()
    docs = set(np.linspace(0,len(df_tweets)-1, len(df_tweets)))
    for term in query:
    ## START DODE
        try:
            # store in termDocs the ids of the docs that contain "term"                        
            termDocs=[posting[0] for posting in index[term]]
            # docs = docs Union termDocs
            docs = docs.intersection(termDocs)
        except:
            #term is not in index
            pass
    docs=list(docs)
    ranked_docs = rankDocuments(query, docs, index, idf, tf, titleIndex)   
    return ranked_docs

In [18]:
print("Insert your query:\n")
query = input()
ranked_docs = search_tf_idf(query, index)    
top = 20

df_tweets.loc[ranked_docs[:top]][['text', 'user', 'created_at', 'hashtags' , 'favorite_count', 'retweet_count', 'source']]

Insert your query:

trump


,text,user,created_at,hashtags,favorite_count,retweet_count,source
217832,"They should all be jailed - Trump, Conway, Iva...","{'id': 27208882, 'id_str': '27208882', 'name':...",Thu Dec 17 16:38:27 +0000 2020,Covid,0,0,"<a href=""http://twitter.com/download/iphone"" r..."
217682,@rambosnack @deronium @kaia198312 @realDonaldT...,"{'id': 322803604, 'id_str': '322803604', 'name...",Thu Dec 17 16:38:24 +0000 2020,None,0,0,"<a href=""http://twitter.com/download/android"" ..."
217619,Trump has tweeted about TV ratings four times ...,"{'id': 16417977, 'id_str': '16417977', 'name':...",Thu Dec 17 16:38:22 +0000 2020,None,0,0,"<a href=""https://mobile.twitter.com"" rel=""nofo..."
217615,via ⁦@nytimes⁩ @cspanwj \nTrump leaves office ...,"{'id': 39504336, 'id_str': '39504336', 'name':...",Thu Dec 17 16:38:22 +0000 2020,Cult45KILLSjobs,0,0,"<a href=""http://twitter.com/download/iphone"" r..."
217597,@LindseyGrahamSC $600 is an insult. Why haven'...,"{'id': 1331645605698007045, 'id_str': '1331645...",Thu Dec 17 16:38:21 +0000 2020,None,0,0,"<a href=""http://twitter.com/download/android"" ..."
217532,"over 300,000 dead from the V, 10 inmates execu...","{'id': 1325112850818404356, 'id_str': '1325112...",Thu Dec 17 16:38:20 +0000 2020,None,0,0,"<a href=""https://mobile.twitter.com"" rel=""nofo..."
216934,I like people who did NOT kiss Trump's ass.,"{'id': 32655131, 'id_str': '32655131', 'name':...",Thu Dec 17 16:38:04 +0000 2020,None,0,0,"<a href=""https://mobile.twitter.com"" rel=""nofo..."
216890,If Trump would of done a Tom Cruise at the beg...,"{'id': 1324681791383457793, 'id_str': '1324681...",Thu Dec 17 16:38:03 +0000 2020,None,0,0,"<a href=""http://twitter.com/download/iphone"" r..."
216620,@marcorubio OMG - you are a hypocrite and a ba...,"{'id': 1158829103614746625, 'id_str': '1158829...",Thu Dec 17 16:37:56 +0000 2020,None,0,0,"<a href=""https://mobile.twitter.com"" rel=""nofo..."
216456,"@eugenegu @realDonaldTrump But, but MSM said T...","{'id': 39924112, 'id_str': '39924112', 'name':...",Thu Dec 17 16:37:52 +0000 2020,None,0,0,"<a href=""http://twitter.com/download/android"" ..."


In [24]:
def search_using_new_ranking(query, index):
    '''
    output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    '''
    query=clean_text(query)
    #docs=set()
    docs = set()
    for term in query:
    ## START DODE
        try:
            # store in termDocs the ids of the docs that contain "term"                        
            termDocs=[posting[0] for posting in index[term]]
            # docs = docs Union termDocs
            docs = docs.union(termDocs)
        except:
            #term is not in index
            pass
    docs=list(docs)
    ranked_docs = rankDocuments(query, docs, index, idf, tf, titleIndex)   
    return ranked_docs

In [26]:
print("Insert your query:\n")
query = input()
ranked_docs = search_tf_idf(query, index)    
top = 1

query = df_tweets.loc[ranked_docs[:top]]['text'].values
ranked_docs = search_using_new_ranking(query[0], index)    
top = 20

df_tweets.loc[ranked_docs[:top]][['text', 'user', 'created_at', 'hashtags' , 'favorite_count', 'retweet_count', 'source']]

Insert your query:

covid


,text,user,created_at,hashtags,favorite_count,retweet_count,source
217856,@CarrieeeeC They're having panic attacks over ...,"{'id': 23965688, 'id_str': '23965688', 'name':...",Thu Dec 17 16:38:28 +0000 2020,None,0,0,"<a href=""http://twitter.com/download/android"" ..."
217837,What if I told you we already have the vaccine...,"{'id': 972294050232532997, 'id_str': '97229405...",Thu Dec 17 16:38:27 +0000 2020,None,0,0,"<a href=""https://mobile.twitter.com"" rel=""nofo..."
217836,The vaccine gonna be $600 😭,"{'id': 3741790334, 'id_str': '3741790334', 'na...",Thu Dec 17 16:38:27 +0000 2020,None,0,0,"<a href=""http://twitter.com/download/iphone"" r..."
217831,We deserve royalties! We paid for the vaccine ...,"{'id': 1008411133458436096, 'id_str': '1008411...",Thu Dec 17 16:38:27 +0000 2020,None,0,0,"<a href=""http://twitter.com/download/android"" ..."
217794,@realDonaldTrump People are dying because of t...,"{'id': 1095880612634918912, 'id_str': '1095880...",Thu Dec 17 16:38:26 +0000 2020,None,0,0,"<a href=""http://twitter.com/download/android"" ..."
217783,"Some vaccine doses kept too cold, Pfizer havin...","{'id': 360024367, 'id_str': '360024367', 'name...",Thu Dec 17 16:38:26 +0000 2020,None,0,0,"<a href=""http://publicize.wp.com/"" rel=""nofoll..."
217770,Give me an update on the H1N1 VACCINE. Stand u...,"{'id': 278770541, 'id_str': '278770541', 'name...",Thu Dec 17 16:38:25 +0000 2020,None,0,0,"<a href=""http://twitter.com/download/android"" ..."
217769,@NathanLerner @Timcast Apparently the people g...,"{'id': 1321166514188357633, 'id_str': '1321166...",Thu Dec 17 16:38:25 +0000 2020,None,0,0,"<a href=""http://twitter.com/download/iphone"" r..."
217764,my uncle tried to compare taking a plane to ta...,"{'id': 853405744514965504, 'id_str': '85340574...",Thu Dec 17 16:38:25 +0000 2020,None,0,0,"<a href=""http://twitter.com/download/iphone"" r..."
217738,Africa is aiming to vaccinate up to 60% of its...,"{'id': 141163282, 'id_str': '141163282', 'name...",Thu Dec 17 16:38:25 +0000 2020,None,0,0,"<a href=""https://about.twitter.com/products/tw..."
